## FIRST APPROACH - MAX-DEPTH OF THE DECISION TREE CLASSIFIER IS NOT OPTIMAL

In [110]:
# Import Libraries
import pandas as pd
from sklearn import tree
from sklearn import metrics

In [111]:
# Load data
df = pd.read_csv("winequality-red.csv")

In [112]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [113]:
# Describe data
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599.0,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free sulfur dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total sulfur dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


In [114]:
# Datatypes in the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [115]:
#Missing values
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [116]:
df['quality'].value_counts()
# This datasetconsists of only six types of quality values.
# We will thus map all quality values from 0 to 5

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

In [117]:
# a mapping dictionary that maps the quality values from 0 to 5
quality_mapping = {
    3: 0,
    4: 1,
    5: 2,
    6: 3,
    7: 4,
    8: 5
}

In [118]:
# use the map function of pandas with any dictionary to convert the values in a given
# column to values in the dictionary
df.loc['quality'] = df['quality'].map(quality_mapping)

In [119]:
df['quality']

0          5.0
1          5.0
2          5.0
3          6.0
4          5.0
          ... 
1595       6.0
1596       6.0
1597       5.0
1598       6.0
quality    NaN
Name: quality, Length: 1600, dtype: float64

In [120]:
# Drop missing values
df.dropna(axis=0, inplace=True)

In [121]:
df.shape

(1599, 12)

In [122]:
# Data splitting
# Shuffle the data before splitting to prevent any bias that might occur from the original ordering of the data
# frac=1 means the entire DataFrame will be shuffled. Each row has an equal chance of being selected.
df = df.sample(frac=1).reset_index(drop=True)

# Top 1000 rows to be used for training
df_train = df.head(1000)

# Bottom 599 rows to be used for testing/validation
df_test = df.tail(599)

In [123]:
# We will now train a decision tree model on the training set.
# Initialize decision tree classifier with a max_depth of 3
clf = tree.DecisionTreeClassifier(max_depth=3)
# choose the columns you want to train the model on (features)
cols = ['fixed acidity', 
 'volatile acidity', 
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [124]:
# Train the model on the provided features and mapped quality from before
clf.fit(df_train[cols], df_train.quality)

DecisionTreeClassifier(max_depth=3)

In [128]:
# Testing the accuracy of the model on the training set and test set
# Generate predictions on the training set
train_predictions = clf.predict(df_train[cols])

# Generate predictions on the testing set
test_predictions = clf.predict(df_test[cols])

# Calculate the accuracy of predictions on the training data set
train_accuracy = metrics.accuracy_score(df_train.quality, train_predictions)

# Calculate the accuracy of predictions on the test data set
test_accuracy = metrics.accuracy_score(df_test.quality, test_predictions)
print(f"Train and Test acuracy with a max depth of 3: {train_accuracy}, {test_accuracy}")

Train and Test acuracy with a max depth of 3: 0.618, 0.5459098497495827


In [129]:
# Increase max_depth to 7
clf = tree.DecisionTreeClassifier(max_depth=7)

# Train the model again
clf.fit(df_train[cols], df_train.quality)

DecisionTreeClassifier(max_depth=7)

In [130]:
# Calculate Accuracy when max depth is 7
# Testing the accuracy of the model on the training set and test set
# Generate predictions on the training set
train_predictions = clf.predict(df_train[cols])

# Generate predictions on the testing set
test_predictions = clf.predict(df_test[cols])

# Calculate the accuracy of predictions on the training data set
train_accuracy = metrics.accuracy_score(df_train.quality, train_predictions)

# Calculate the accuracy of predictions on the test data set
test_accuracy = metrics.accuracy_score(df_test.quality, test_predictions)
print(f"Train and Test acuracy with a max depth of 3: {train_accuracy}, {test_accuracy}")

Train and Test acuracy with a max depth of 3: 0.777, 0.5442404006677797
